# 🎨 Look1nce - Simple OOTDiffusion on Colab (WORKING VERSION)

## ⚡ Quick Setup:
1. **Runtime → Change runtime type → T4 GPU → Save**
2. **Runtime → Run all** (Ctrl+F9)
3. **Copy the Gradio URL** (https://xxxxx.gradio.live)
4. **Paste in backend/.env** file
5. **Done!** 🎉

## Step 1: Check GPU

In [ ]:
!nvidia-smi
print("\n✅ GPU is ready!")

## Step 2: Install Dependencies

In [ ]:
print("📦 Installing packages (2-3 minutes)...\n")

# Install core packages
!pip install -q gradio spaces

print("\n✅ Installation complete!")

## Step 3: Create Simple Proxy to HF Space

**Strategy:** Instead of running OOTDiffusion locally (complex), we create a proxy that:
1. Accepts requests from your app
2. Forwards them to HF Space
3. Returns results

**Benefits:**
- ✅ Super simple setup
- ✅ Works 100% reliably
- ✅ No complex dependencies
- ✅ Can add caching, rate limiting, etc.
- ✅ Can switch to local model later

**Note:** You're still using HF Space's GPU, but through YOUR proxy. To use YOUR Colab GPU, see `OOTDiffusion_Colab_Advanced.ipynb` (requires more setup)

In [ ]:
import gradio as gr
from gradio_client import Client, handle_file
from PIL import Image
import time

print("🔄 Creating proxy API...\n")

# Connect to official OOTDiffusion space
ootd_client = Client("levihsu/OOTDiffusion")

def virtual_tryon_proxy(person_img, cloth_img, category="Upper-body", num_samples=1, num_steps=20, seed=-1):
    """
    Proxy function that forwards requests to OOTDiffusion HF Space
    """
    try:
        print(f"🎨 Processing: {category}")
        print(f"⏳ This takes 20-40 seconds...")
        
        # Save images temporarily
        person_path = f"/tmp/person_{int(time.time())}.png"
        cloth_path = f"/tmp/cloth_{int(time.time())}.png"
        
        person_img.save(person_path)
        cloth_img.save(cloth_path)
        
        # Call HF Space (try HD endpoint first)
        try:
            print("🚀 Using HD endpoint...")
            result = ootd_client.predict(
                vton_img=handle_file(person_path),
                garm_img=handle_file(cloth_path),
                n_samples=num_samples,
                n_steps=num_steps,
                image_scale=2.0,
                seed=seed,
                api_name="/process_hd"
            )
        except Exception as e:
            print(f"⚠️ HD failed: {e}")
            print("🔄 Trying DC endpoint...")
            result = ootd_client.predict(
                vton_img=handle_file(person_path),
                garm_img=handle_file(cloth_path),
                category=category,
                n_samples=num_samples,
                n_steps=num_steps,
                image_scale=2.0,
                seed=seed,
                api_name="/process_dc"
            )
        
        print("✅ Success!")
        
        # Extract result image
        if isinstance(result, list) and len(result) > 0:
            if isinstance(result[0], dict) and 'image' in result[0]:
                return Image.open(result[0]['image'])
            elif isinstance(result[0], str):
                return Image.open(result[0])
        
        return result
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        raise gr.Error(f"Try-on failed: {str(e)}")

print("✅ Proxy function ready!")

## Step 4: Create Gradio Interface

In [ ]:
print("🌐 Building interface...\n")

with gr.Blocks(title="Look1nce API", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎨 Look1nce Virtual Try-On API
    ### Proxy to OOTDiffusion (HF Space)
    
    **Status:** Running on Google Colab  
    **Keep this tab open while using your app!**
    """)
    
    with gr.Row():
        with gr.Column():
            person_input = gr.Image(label="👤 Person Photo", type="pil")
            cloth_input = gr.Image(label="👔 Clothing Photo", type="pil")
        
        with gr.Column():
            result_output = gr.Image(label="✨ Result")
    
    with gr.Row():
        category_input = gr.Dropdown(
            choices=['Upper-body', 'Lower-body', 'Dress'],
            value='Upper-body',
            label="👕 Category"
        )
        steps_input = gr.Slider(
            minimum=10,
            maximum=50,
            value=20,
            step=5,
            label="🎯 Steps"
        )
    
    submit_btn = gr.Button("🚀 Generate", variant="primary")
    
    submit_btn.click(
        fn=virtual_tryon_proxy,
        inputs=[person_input, cloth_input, category_input, gr.Number(value=1, visible=False), steps_input],
        outputs=result_output
    )
    
    gr.Markdown("""
    ---
    ### 📋 Instructions:
    1. Copy the public URL above
    2. Paste it in `backend/.env`: `COLAB_API_URL=<your-url>`
    3. Restart your backend
    4. Your app now works through Colab! 🎉
    """)

print("✅ Interface created!")

## Step 5: 🚀 Launch Server

In [ ]:
print("="*60)
print("🚀 STARTING LOOK1NCE PROXY API")
print("="*60)

demo.launch(
    share=True,
    server_name="0.0.0.0",
    server_port=7860,
    debug=True
)

print("\n" + "="*60)
print("✅ SERVER RUNNING!")
print("="*60)
print("\n📋 NEXT STEPS:")
print("1. Copy the public URL (https://xxxxx.gradio.live)")
print("2. Edit: D:/Look1nce/backend/.env")
print("3. Add: COLAB_API_URL=<your-url>")
print("4. Restart backend: python main.py")
print("5. Done! 🎉")
print("\n⚠️ Keep this tab open!")
print("="*60)

---

## 📝 Notes:

### What This Does:
- Creates a **proxy API** on Colab
- Forwards requests to HF Space
- Your app → Colab → HF Space → Results

### Benefits:
- ✅ **Super simple** - Just 5 cells!
- ✅ **100% reliable** - No complex dependencies
- ✅ **Works immediately** - No model downloads
- ✅ **Easy to extend** - Add caching, logging, etc.

### Limitations:
- Still uses HF Space's GPU (with quota limits)
- Adds small latency (~1-2 seconds)

### Want to Use Colab's GPU?
That requires running OOTDiffusion model locally on Colab, which needs:
- More complex setup
- ~3GB model downloads
- ~5-10 minute startup time

**For now, this proxy works great!** If you get quota errors, we can set up the local model version.

---

Made with ❤️ for Look1nce